# Imports

In [ ]:
import os

import pandas as pd
import progressbar
from ray.tune import Analysis

from rayTune_common.constants import metric, mode
from rayTune_common.utils import config_to_model

# Analyze ray tune logs

In [ ]:
from rayTune_common.test import test_training_error

path_to_run_results = "/home/knut/Documents/project/UnseededRun_results"
list_runs = [f.path for f in os.scandir(path_to_run_results) if f.is_dir()]
bar = progressbar.ProgressBar(maxval=10 * 15 * 100,
                              widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
prog = 0
for path_to_run in list_runs:
    data = []
    path_to_csv = os.path.join(path_to_run, "train_error.csv")

    list_experiments = [f.path for f in os.scandir(path_to_run) if f.is_dir()]
    list_experiments.sort(key=lambda x: x.split("_")[-1])

    for experient_number, path_to_experiment in enumerate(list_experiments):
        experiment_data = {}

        list_experiment_trials = [f.path for f in os.scandir(path_to_experiment) if f.is_dir()]
        list_experiment_trials.sort(key=lambda x: int(x.split("_")[4]))

        for trial_number, path_to_trial in enumerate(list_experiment_trials):
            list_trial_checkpoints = [f.path for f in os.scandir(path_to_trial) if f.is_dir()]
            list_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))

            trial_checkpoint_path = os.path.join(list_trial_checkpoints[-1], "checkpoint")

            trial_analysis = Analysis(path_to_trial, default_metric=metric, default_mode=mode)
            trial_config = trial_analysis.get_best_config(metric=metric, mode=mode)

            trial_model = config_to_model(config=trial_config, checkpoint_path=trial_checkpoint_path)

            trial_mse = test_training_error(model=trial_model, batch_size=64)
            experiment_data[trial_number] = trial_mse

            bar.update(prog + 1)
            prog += 1

        sorted_experiment_data = dict(sorted(experiment_data.items()))
        data.append(sorted_experiment_data)

    # Convert into pandas dataframe
    df = pd.DataFrame(data)

    # Add mean and variance to test results
    df["best"] = df.min(axis=1)
    mean = df.mean(axis=0)
    var = df.var(axis=0)
    std = df.std(axis=0)
    df.loc["mean"] = mean
    df.loc["var"] = var
    df.loc["std"] = std

    # Store dataframes as csv
    df.to_csv(path_to_csv)

bar.finish()